In [4]:
import cv2
import numpy as np
import os
from collections import Counter
import math
from PIL import Image

In [2]:
def contar_ocurrencias_pixeles(imagen, umbral=128):
    # Cargar la imagen
    img = cv2.imread(imagen, cv2.IMREAD_GRAYSCALE)
    _, binaria = cv2.threshold(img, umbral, 255, cv2.THRESH_BINARY)

    # Obtener los valores de los píxeles
    valores_pixeles = binaria.flatten()

    # Contar las ocurrencias de cada valor de píxel
    contador_ocurrencias = Counter(valores_pixeles)

    return contador_ocurrencias

# Ruta de la carpeta con las imágenes
carpeta_imagenes = "ground_truths_w"

# Número total de imágenes
num_imagenes = 6

# Lista para almacenar los diccionarios de ocurrencias de cada imagen
lista_diccionarios = []

# Iterar sobre todas las imágenes
for numero_imagen in range(1, num_imagenes + 1):
    # Obtener la ruta de la imagen
    ruta_imagen = f"{carpeta_imagenes}/{numero_imagen}.png"
    
    # Obtener el diccionario de ocurrencias de píxeles para la imagen actual
    diccionario_ocurrencias = contar_ocurrencias_pixeles(ruta_imagen)
    
    # Guardar el diccionario en la lista
    lista_diccionarios.append(diccionario_ocurrencias)
    

# Ruta de la carpeta con las imágenes
carpeta_imagenes = "5-final"

# Número total de imágenes
num_imagenes = 6

# Lista para almacenar los diccionarios de ocurrencias de cada imagen
lista_diccionarios2 = []

# Iterar sobre todas las imágenes
for numero_imagen in range(1, num_imagenes + 1):
    # Obtener la ruta de la imagen
    ruta_imagen = f"{carpeta_imagenes}/{numero_imagen}.png"
    
    # Obtener el diccionario de ocurrencias de píxeles para la imagen actual
    diccionario_ocurrencias = contar_ocurrencias_pixeles(ruta_imagen)
    
    # Guardar el diccionario en la lista
    lista_diccionarios2.append(diccionario_ocurrencias)

In [3]:
for i in range(0,6):
    print(f"Ground Truth {i} " + f'{lista_diccionarios[i]}')
    print(f"Segmentada   {i} " + f'{lista_diccionarios2[i]}')

Ground Truth 0 Counter({0: 4586194, 255: 329006})
Segmentada   0 Counter({0: 4580973, 255: 334227})
Ground Truth 1 Counter({0: 4051986, 255: 863214})
Segmentada   1 Counter({0: 4064760, 255: 850440})
Ground Truth 2 Counter({0: 3145403, 255: 1769797})
Segmentada   2 Counter({0: 3150968, 255: 1764232})
Ground Truth 3 Counter({0: 4691303, 255: 223897})
Segmentada   3 Counter({0: 4686179, 255: 229021})
Ground Truth 4 Counter({0: 4176561, 255: 738639})
Segmentada   4 Counter({0: 4153606, 255: 761594})
Ground Truth 5 Counter({0: 2817729, 255: 2097471})
Segmentada   5 Counter({0: 2804752, 255: 2110448})


In [4]:
def calcular_metricas_opencv(segmentacion_path, ground_truth_path):
    # Cargar las imágenes
    segmentacion = cv2.imread(segmentacion_path, cv2.IMREAD_GRAYSCALE)
    ground_truth = cv2.imread(ground_truth_path, cv2.IMREAD_GRAYSCALE)

    # Convertir a imágenes binarias (0 o 255) si no lo están
    _, segmentacion = cv2.threshold(segmentacion, 127, 255, cv2.THRESH_BINARY)
    _, ground_truth = cv2.threshold(ground_truth, 127, 255, cv2.THRESH_BINARY)

    # Calcular métricas

    # Verdaderos positivos: Aquellos que son positivos en ambos
    VP = np.sum(np.logical_and(segmentacion == 255, ground_truth == 255))

    # Falsos positivos: Aquellos que son positivo en segmentacion, falso en ground truth
    FP = np.sum(np.logical_and(segmentacion == 255, ground_truth == 0))

    # Verdaderos positivos: quellos que son negativos en ambos
    VN = np.sum(np.logical_and(segmentacion == 0, ground_truth == 0))

    # Falsos negativos: Aquellos que son falso en segmentacion, positivo en ground truth
    FN = np.sum(np.logical_and(segmentacion == 0, ground_truth == 255))

    precision = VP / (VP + FP) if (VP + FP) != 0 else 0

    sensibilidad = VP / (VP + FN) if (VP + FN) != 0 else 0

    especificidad = VN / (VN + FP) if (VN + FP) != 0 else 0

    DSC = (2 * VP) / ( 2 * VP + FP + FN)

    Similitud = 1 - ( math.sqrt( (1-precision)**2 + (1-sensibilidad)**2 ) / math.sqrt(2))

    # Descomentar para ver Positivos y negativos, falsos y verdaderos de cada imagen
    print(F"VP: {VP}, FP: {FP}, VN: {VN}, FN: {FN}")
    print(f'Precision: {precision*100}%')
    print(f'Sensibilidad: {sensibilidad*100}%')
    print(f'Especificidad: {especificidad*100}%')
    print(f'Similaridad de Dice: {DSC}')
    print(f'Similitud: {Similitud}')

    

# Ejemplo de uso - Solo hay que cambiar el 1 por el numero de la imagen a comparar:
segmentacion_path = "5-final/6.png"
ground_truth_path = "ground_truths_w/6.png"

calcular_metricas_opencv(segmentacion_path, ground_truth_path)

VP: 2084986, FP: 25462, VN: 2792267, FN: 12485
Precision: 98.79352630341994%
Sensibilidad: 99.40475935066564%
Especificidad: 99.09636448359655%
Similaridad de Dice: 0.9909820032182178
Similitud: 0.9904871397278223


In [5]:
def recortar_redimensionar_imagen(input_path, output_path, nuevo_tamano):
    # Abrir la imagen
    imagen = Image.open(input_path)

    # Obtener el tamaño actual de la imagen
    ancho_actual, alto_actual = imagen.size

    # Calcular las coordenadas para recortar centrado
    izquierda = (ancho_actual - nuevo_tamano[0]) // 2 + 10
    arriba = (alto_actual - nuevo_tamano[1]) // 2 + 10
    derecha = (ancho_actual + nuevo_tamano[0]) // 2 + 10
    abajo = (alto_actual + nuevo_tamano[1]) // 2 + 10

    # Recortar la imagen
    imagen_recortada = imagen.crop((izquierda, arriba, derecha, abajo))

    # Redimensionar la imagen
    imagen_redimensionada = imagen_recortada.resize(nuevo_tamano, Image.ANTIALIAS)

    # Guardar la imagen resultante
    imagen_redimensionada.save(output_path)

def batch_process_images(input_folder, output_folder, nuevo_tamano):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # List all files in the input folder
    files = os.listdir(input_folder)

    # Process each image in the folder
    for file in files:
        if file.endswith(".png") or file.endswith(".jpg") or file.endswith(".jpeg"):
            # Construct full paths for input and output
            input_path = os.path.join(input_folder, file)
            output_path = os.path.join(output_folder, file)

            # Call the function to process the image
            recortar_redimensionar_imagen(input_path, output_path, nuevo_tamano)

if __name__ == "__main__":
    # Set the input folder, output folder, and nuevo_tamano
    input_folder = "ground_truths_w"
    output_folder = "ground_truths_w_r"
    nuevo_tamano = (2250, 1830)

    # Call the batch processing function
    batch_process_images(input_folder, output_folder, nuevo_tamano)

C:\Users\Javier\AppData\Local\Temp\ipykernel_21004\1378881234.py:18: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  imagen_redimensionada = imagen_recortada.resize(nuevo_tamano, Image.ANTIALIAS)


In [7]:
def contar_ocurrencias_pixeles(imagen, umbral=128):
    # Cargar la imagen
    img = cv2.imread(imagen, cv2.IMREAD_GRAYSCALE)
    _, binaria = cv2.threshold(img, umbral, 255, cv2.THRESH_BINARY)

    # Obtener los valores de los píxeles
    valores_pixeles = binaria.flatten()

    # Contar las ocurrencias de cada valor de píxel
    contador_ocurrencias = Counter(valores_pixeles)

    return contador_ocurrencias

# Ruta de la carpeta con las imágenes
carpeta_imagenes = "ground_truths_w_r"

# Número total de imágenes
num_imagenes = 6

# Lista para almacenar los diccionarios de ocurrencias de cada imagen
lista_diccionarios = []

# Iterar sobre todas las imágenes
for numero_imagen in range(1, num_imagenes + 1):
    # Obtener la ruta de la imagen
    ruta_imagen = f"{carpeta_imagenes}/{numero_imagen}.png"
    
    # Obtener el diccionario de ocurrencias de píxeles para la imagen actual
    diccionario_ocurrencias = contar_ocurrencias_pixeles(ruta_imagen)
    
    # Guardar el diccionario en la lista
    lista_diccionarios.append(diccionario_ocurrencias)
    

# Ruta de la carpeta con las imágenes
carpeta_imagenes = "9-final"

# Número total de imágenes
num_imagenes = 6

# Lista para almacenar los diccionarios de ocurrencias de cada imagen
lista_diccionarios2 = []

# Iterar sobre todas las imágenes
for numero_imagen in range(1, num_imagenes + 1):
    # Obtener la ruta de la imagen
    ruta_imagen = f"{carpeta_imagenes}/{numero_imagen}.png"
    
    # Obtener el diccionario de ocurrencias de píxeles para la imagen actual
    diccionario_ocurrencias = contar_ocurrencias_pixeles(ruta_imagen)
    
    # Guardar el diccionario en la lista
    lista_diccionarios2.append(diccionario_ocurrencias)

In [8]:
for i in range(0,6):
    print(f"Ground Truth {i} " + f'{lista_diccionarios[i]}')
    print(f"Segmentada   {i} " + f'{lista_diccionarios2[i]}')

Ground Truth 0 Counter({0: 3788494, 255: 329006})
Segmentada   0 Counter({0: 4044695, 255: 72805})
Ground Truth 1 Counter({0: 3254590, 255: 862910})
Segmentada   1 Counter({0: 3915718, 255: 201782})
Ground Truth 2 Counter({0: 2380804, 255: 1736696})
Segmentada   2 Counter({0: 3814843, 255: 302657})
Ground Truth 3 Counter({0: 3893603, 255: 223897})
Segmentada   3 Counter({0: 4052607, 255: 64893})
Ground Truth 4 Counter({0: 3378861, 255: 738639})
Segmentada   4 Counter({0: 3881348, 255: 236152})
Ground Truth 5 Counter({0: 2079881, 255: 2037619})
Segmentada   5 Counter({0: 3973515, 255: 143985})


In [18]:
def calcular_metricas_opencv(segmentacion_path, ground_truth_path):
    # Cargar las imágenes
    segmentacion = cv2.imread(segmentacion_path, cv2.IMREAD_GRAYSCALE)
    ground_truth = cv2.imread(ground_truth_path, cv2.IMREAD_GRAYSCALE)

    # Convertir a imágenes binarias (0 o 255) si no lo están
    _, segmentacion = cv2.threshold(segmentacion, 127, 255, cv2.THRESH_BINARY)
    _, ground_truth = cv2.threshold(ground_truth, 127, 255, cv2.THRESH_BINARY)

    # Calcular métricas

    # Verdaderos positivos: Aquellos que son positivos en ambos
    VP = np.sum(np.logical_and(segmentacion == 255, ground_truth == 255))

    # Falsos positivos: Aquellos que son positivo en segmentacion, falso en ground truth
    FP = np.sum(np.logical_and(segmentacion == 255, ground_truth == 0))

    # Verdaderos positivos: quellos que son negativos en ambos
    VN = np.sum(np.logical_and(segmentacion == 0, ground_truth == 0))

    # Falsos negativos: Aquellos que son falso en segmentacion, positivo en ground truth
    FN = np.sum(np.logical_and(segmentacion == 0, ground_truth == 255))

    precision = VP / (VP + FP) if (VP + FP) != 0 else 0

    sensibilidad = VP / (VP + FN) if (VP + FN) != 0 else 0

    especificidad = VN / (VN + FP) if (VN + FP) != 0 else 0

    DSC = (2 * VP) / ( 2 * VP + FP + FN)

    Similitud = 1 - ( math.sqrt( (1-precision)**2 + (1-sensibilidad)**2 ) / math.sqrt(2))

    # Descomentar para ver Positivos y negativos, falsos y verdaderos de cada imagen
    print(F"VP: {VP}, FP: {FP}, VN: {VN}, FN: {FN}")
    print(f'Precision: {precision*100}%')
    print(f'Sensibilidad: {sensibilidad*100}%')
    print(f'Especificidad: {especificidad*100}%')
    print(f'Similaridad de Dice: {DSC}')
    print(f'Similitud: {Similitud}')

    

# Ejemplo de uso - Solo hay que cambiar el 1 por el numero de la imagen a comparar:
segmentacion_path = "9-final/5.png"
ground_truth_path = "ground_truths_w_r/5.png"

calcular_metricas_opencv(segmentacion_path, ground_truth_path)

VP: 235304, FP: 5180, VN: 3373681, FN: 503335
Precision: 97.8460105454001%
Sensibilidad: 31.856427835519106%
Especificidad: 99.84669390069612%
Similaridad de Dice: 0.48064237077466265
Similitud: 0.5179115175662075
